In [10]:
import os
import pefile
import pprint as pp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import csv
import glob
import magic
import hashlib
import sys
import struct
import peutils
import pefileHuman as peh

In [11]:
# def sha256_checksum(filename, block_size=65536):
#     sha256 = hashlib.sha256()
#     with open(filename, 'rb') as f:
#         for block in iter(lambda: f.read(block_size), b''):
#             sha256.update(block)
#     return sha256.hexdigest()



Now we move on to write a small method that constructs a dictionnary for each PE File thus each sample will be represented as a python dictionnary where keys are the features and values are the value of each parsed field .

In [12]:
def pe2vec(directory):
    """
    dirty function (handling all exceptions) for each sample
    it construct a dictionary of dictionaries in the format:
    sample x : pe informations
    """
    dataset = {}
    #directory = "./data/"
    print("")
    print("[*] Extracting the PE file data: ")
    print("")
    for subdir, dirs, files in os.walk(directory):
        for f in files:
            file_path = os.path.join(subdir, f)
            
            #print(magic.from_file(file_path))
            if re.match('^PE.*', magic.from_file(file_path)):
                try:
                    #print("[+] "+file_path)
                    pe = peh.PEFile(file_path)
                    dataset[str(f)] = pe.Construct()
                except Exception as e:
                    raise
            else:
                #print()
                #print("[-] File not PE: "+file_path+" \n=>\t "+magic.from_file(file_path))
                #print()
                #print("Removing file.")
                os.remove(file_path)
    return dataset
    
    

### Testing the pe2vec() method and PEFile class

Print out the dataset for the Malware and Clean Samples


In [13]:
# for subdir, dirs, files in os.walk("./data/check/"):
#         for f in files:
#             #print(f)
#             os.rename("./data/check/" + f, "./data/check/" + sha256_checksum("./data/check/" + f))
            
# print("Done Renaming files to sha256")

In [14]:
# create a dataset dictionary from the collected PE file information
# we might consider adding an MD5 hash value and aappend it to a new column
# an asscoiated MD5 value could be used to lookup the VirusToltal score to confirm it is Malware if we did not get the sample from a good source
ds_check = {}
ds_check = pe2vec("./data/check/")
print("")
print("[*] PE information:")
print("")
#pp.pprint(ds_malware)

print("")
print("[*] Completed PE information extraction:")
print("")


[*] Extracting the PE file data: 


[*] PE information:


[*] Completed PE information extraction:



Loop trough all samples in a folder and process each one of them then dump all those dictionaries into a csv file that we will use .

In [15]:
# now that we have a dictionary let's put it in a clean csv file
def vec2csv(dataset, output_file):
    df = pd.DataFrame(dataset)
    test_data = df.transpose()  # transpose to have the features as columns and samples as rows
# utf-8 is prefered 
#output_file = './output/dataset.csv'
    test_data.to_csv(output_file,sep=',', encoding='utf-8')
    print("")
    print("[+] Saving file to: " + output_file)
    print("")
        
        

### Export the dataset as a csv file

In [16]:
datasetOutput_malware = "./dataset/dataset_check.csv"
vec2csv(ds_check, datasetOutput_malware)


[+] Saving file to: ./dataset/dataset_check.csv



Use Magic %matplotlib to display graphics inline instead of in a popup window.



In [17]:
%matplotlib inline



#### Check Dataframe

In [18]:
# get the dataframe
df_check = pd.read_csv("./dataset/dataset_check.csv")
df_check.shape
df_check.head(5)



,Unnamed: 0,filename,e_magic,e_cblp,e_cp,e_crlc,e_cparhdr,e_minalloc,e_maxalloc,e_ss,...,IATSize,DelayImportRVA,DelayImportSize,COMDescriptorRVA,COMDescriptorSize,ImportedDLL,NumberOfImportDLL,SectionNames,NumberOfImportFunctions,ImportedFunctions
0,2a389c51349d6c3f0180538b64e1939fc8cff79fa3a2aa...,./data/check/2a389c51349d6c3f0180538b64e1939fc...,23117,144,3,0,4,0,65535,0,...,80,0,0,0,0,"{'USER32.dll': 'MoveWindow', 'KERNEL32.dll': '...",2,"{'.text\x00\x00\x00': 142336, '.data\x00\x00\x...",18,"['OpenIcon', 'CreatePopupMenu', 'CloseWindow',..."
1,2a38b2017c26f04b2deab479be2c0ea011b548d9f2ff30...,./data/check/2a38b2017c26f04b2deab479be2c0ea01...,23117,144,3,0,4,0,65535,0,...,484,0,0,0,0,"{'WS2_32.dll': 'inet_addr', 'RPCRT4.dll': 'Uui...",7,"{'.text\x00\x00\x00': 143360, '.rdata\x00\x00'...",114,"['htonl', 'ntohl', 'inet_addr', 'UuidFromStrin..."
2,2a44946b461e2435e77add694c1e4f625e6c8cec561813...,./data/check/2a44946b461e2435e77add694c1e4f625...,23117,144,3,0,4,0,65535,0,...,108,0,0,0,0,"{'ntoskrnl.exe': 'RtlUnwind', 'HAL.dll': 'KeGe...",2,"{'.text\x00\x00\x00': 59904, '.rdata\x00\x00':...",25,"['RtlInitUnicodeString', '_allmul', '_alldiv',..."
3,2a496265df0d56fd40522f5f392734b920cd3513a9ef6a...,./data/check/2a496265df0d56fd40522f5f392734b92...,23117,144,3,0,4,0,65535,0,...,108,0,0,0,0,"{'ntoskrnl.exe': 'RtlUnwind', 'HAL.dll': 'KeGe...",2,"{'.text\x00\x00\x00': 59904, '.rdata\x00\x00':...",25,"['RtlInitUnicodeString', '_allmul', '_alldiv',..."
4,2a5180b99dcf09fd1cf91a53aaa01b97843d942253f8f3...,./data/check/2a5180b99dcf09fd1cf91a53aaa01b978...,23117,144,3,0,4,0,65535,0,...,436,0,0,0,0,"{'COMCTL32.dll': 'ImageList_Remove', 'USER32.d...",5,"{'.text\x00\x00\x00': 163840, '.refee\x00\x00'...",103,"['DestroyPropertySheetPage', 'PropertySheetW',..."
